In [2]:
import hashlib
import csv
import random

## Function to make an input file for the 

In [15]:
def generate_input_file(file_name:str, num_lines=100):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Hash Value'])
        for _ in range(num_lines):
            name = f'User_{random.randint(1, 100)}'
            num = random.randint(1000, 10000)
            hash_value = hashlib.sha256(str(num).encode()).hexdigest()
            writer.writerow([name, hash_value])
generate_input_file('source.csv', 10)

User_13 : 4150
User_6 : 8069
User_49 : 7896
User_30 : 2628
User_77 : 7971
User_57 : 4187
User_10 : 6719
User_12 : 4422
User_12 : 4389
User_59 : 6241


In [16]:
def hash_password_hack(input_file_name, output_file_name):
    # Create an empty dictionary to store the hashed values and their corresponding numbers.
    hashed_dict = {}
    
    # Loop through all numbers from 1000 to 9999.
    for num in range(1000, 10000):
        # Convert the number to a string and compute its hash value using sha256 algorithm.
        hash_value = hashlib.sha256(str(num).encode()).hexdigest()
        
        # Store the hash value and the corresponding number in the dictionary.
        hashed_dict[hash_value] = num
        
    print(hashed_dict)
    # Open the input CSV file in read mode.
    with open(input_file_name, 'r') as input_file:
        # Read the data from the input file into a list of lines.
        next(input_file)  # skip the first line being the header of the file
        lines = input_file.readlines()
        
    # Open the output CSV file in write mode.
    with open(output_file_name, 'w') as output_file:
        # Loop through each line in the input file.
        for line in lines:
            # Split the line into name and hash value.
            name, hash_value = line.strip().split(',')
            
            # Write the name and the corresponding number from the dictionary to the output file.
            output_file.write(f"{name},{hashed_dict[hash_value]}\n")

In [17]:
hash_password_hack('source.csv', 'output.csv')

{'40510175845988f13f6162ed8526f0b09f73384467fa855e1e79b44a56562a58': 1000, 'fe675fe7aaee830b6fed09b64e034f84dcbdaeb429d9cccd4ebb90e15af8dd71': 1001, 'b281bc2c616cb3c3a097215fdc9397ae87e6e06b156cc34e656be7a1a9ce8839': 1002, '8c9a013ab70c0434313e3e881c310b9ff24aff1075255ceede3f2c239c231623': 1003, '75992a5ac67ff644d3063976c2effd10bdd93fcc109798e3d5c1acf2e530d01a': 1004, '7f861bcee185de001377d79e08af62e94b1e7718e2470e08520c917f8d953602': 1005, '478c4ffb1cbcea37956a748e6c19d8eadd0a47e86f5e308d26cad39453b5d1ab': 1006, '2c8b871e52d4e5f5db5ff84a82a45327e20df77edef961c4b6fa0e9c3d97ce5b': 1007, '9aaf689fbcdfe9f64a071f9cbe28ae44193fa218e72af24456f44bed64583b4d': 1008, '6ad4a6b1e5ea5569795e516d71909e0ce4809d9dc983d2c219144f684f816e12': 1009, '7a5df5ffa0dec2228d90b8d0a0f1b0767b748b0a41314c123075b8289e4e053f': 1010, '3dd9c0995d54c0abd51a90f1d57b1ce77bc885fc8a7cea52dcad3c2540dda5ee': 1011, '165940940a02a187e4463ff467090930038c5af8fc26107bf301e714f599a1da': 1012, '2ec42bc1f3e672fc1cf7fdcfc23246415dcf